<a href="https://colab.research.google.com/github/stephenbeckr/ML-theory-class/blob/solutions/LeastSquaresChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.linalg as sla
from timeit import default_timer as timer

# Least squares challenge
Least squares problems are dead simple, so you probably know how to solve them, right?

Below are two challenges, one with a nearly square matric, the other with a very tall matrix. We've constructed the problem so that we know what the answer is (the all ones vector).  Your job is to find the right answer such that:
- you have at least 10 digits of accuracy, e.g., $\|\hat{x}-x_\text{true}\|_\infty \le 10^{-10}$
- Find the answer quickly:
  - For problem 1, do this in under 10 seconds (for both Python/colab and Matlab)
  - For problem 2, do this in under 10 seconds (Matlab) or 5 seconds (Python/colab)

APPM 4490/5490 Theory of Machine Learning, Prof. Becker

In [ ]:
# Problem 1
rng = np.random.default_rng(12345)

n   = int(5e3)
m   = n+1
A       = rng.standard_normal( (m,n) )
xTrue   = np.ones(n)
b       = A@xTrue

In [ ]:
# Problem 2
rng = np.random.default_rng(12345)

n   = int(2e3)
n2  = int(n/2)
m   = int(1e4)
A       = rng.standard_normal( (m,n2) )
A       = np.hstack(  (A,A+1e-4*rng.standard_normal( (m,n2) )) )

xTrue   = np.ones(n)
b       = A@xTrue

## Here's the "bad way" to do it
If you have just some knowledge of linear algebra, this is the most obvious way to solve it. It's actually quite fast (especially for case 2), but not accurate, because you are squaring the condition number
- For problem 1, it takes about 15 seconds but error is only 1e-9
- For problem 2, it's just 2.2 seconds but erorr is 2.9e-6


In [ ]:
start = timer()

x   = np.linalg.inv( A.T @ A ) @ (A.T@b )

end = timer()
print(f'Elapsed time is {end-start:.5f} seconds')
err   = np.linalg.norm(x-xTrue,np.Inf)
print(f'   and l_inf error is {err:.2e}')

#### Variant
If you insist on solving the normal equations, you can at least do it better

In [ ]:
start = timer()

x   = sla.solve( A.T @ A , A.T@b, assume_a='pos' )

end = timer()
print(f'Elapsed time is {end-start:.5f} seconds')
err   = np.linalg.norm(x-xTrue,np.Inf)
print(f'   and l_inf error is {err:.2e}')

## Here's the "standard way" to do it, using builtin solvers

This is a good way, and usually the most accurate.  However, under-the-hood, it does pivoting which is not necessary in our case, and that makes it slow
- For problem 1, it takes up to 70 seconds
- For problem 2 it's faster, taking about 8.5 seconds

In [ ]:
start = timer()

x   = np.linalg.lstsq(A,b, rcond=None)[0] # 70 seconds or so

end = timer()
print(f'Elapsed time is {end-start:.5f} seconds')
err   = np.linalg.norm(x-xTrue,np.Inf)
print(f'   and l_inf error is {err:.2e}')

## And here's the fast way to do it
We'll do a QR decomposition and cancel some terms "by hand". This is faster than `lstsq` since we're not going to pivot
- For problem 1, about 16 seconds and 4e-13 error
- For problem 2, about 7 seconds and 3e-11 error

In [ ]:
start = timer()

Q,R = np.linalg.qr( A, mode="reduced")
Qtb = Q.T@b
x   = sla.solve_triangular(R, Qtb )

end = timer()
print(f'Elapsed time is {end-start:.5f} seconds')
err   = np.linalg.norm(x-xTrue,np.Inf)
print(f'   and l_inf error is {err:.2e}')

#### Variant
This is getting closer to what `lstsq` does "under-the-hood" except we're not going to pivot. When doing the QR decomposition, we don't need the Q matrix explicitly, we only need to apply it to a vector. Scipy has functions that do this for us
- For problem 1, about 8.7 seconds and 2.3e-13 error
- For problem 2, about 4.4 seconds and 1.8e-11 error

In [ ]:
start = timer()

Qtb, R = sla.qr_multiply( A, b, mode="right", pivoting=False )
x      = sla.solve_triangular(R, Qtb )

end = timer()
print(f'Elapsed time is {end-start:.5f} seconds')
err   = np.linalg.norm(x-xTrue,np.Inf)
print(f'   and l_inf error is {err:.2e}')